In [7]:
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urljoin
import pandas as pd
import glob


In [8]:

def download_migration_flows():
   
    # Use current directory for downloads
    output_dir = "."

    # Census.gov URL
    url = "https://www.census.gov/data/tables/time-series/demo/geographic-mobility/state-to-state-migration.html"

    # Get the webpage content
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to access the webpage. Status code: {response.status_code}")
        return

    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links that match our pattern - updated to handle extra spaces
    pattern = r"State-to-State Migration Flows:\s*20(0[5-9]|1[0-9]|2[0-3])"
    
    # Keep track of downloaded files
    downloaded = 0
    skipped = 0

    # Find all links
    for link in soup.find_all('a'):
        text = link.get_text().strip()
        # Skip if the text contains a year range
        if '-' in text and re.search(r'20\d{2}\s*-\s*20\d{2}', text):
            continue
        if re.search(pattern, text):
            # Extract the year using regex - updated to handle extra spaces
            year_match = re.search(r'20(0[5-9]|1[0-9]|2[0-3])', text)
            year = year_match.group(0) if year_match else None
            
            if year:
                # Get the href attribute
                href = link.get('href')
                if href:
                    # Construct full URL
                    file_url = urljoin("https://www.census.gov", href)
                    
                    # Check both possible extensions
                    xlsx_path = os.path.join(output_dir, f"migration_flows_{year}.xlsx")
                    xls_path = os.path.join(output_dir, f"migration_flows_{year}.xls")
                    
                    # Skip if either file exists
                    if os.path.exists(xlsx_path) or os.path.exists(xls_path):
                        print(f"File for year {year} already exists, skipping...")
                        skipped += 1
                        continue
                    
                    try:
                        # Download the file with updated headers
                        headers = {
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                            'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet, application/vnd.ms-excel'
                        }
                        print(f"Downloading file for year {year}...")
                        file_response = requests.get(file_url, headers=headers, allow_redirects=True)
                        
                        # Print debug info
                        print(f"URL: {file_url}")
                        content_type = file_response.headers.get('Content-Type', 'No content type')
                        print(f"Content-Type: {content_type}")
                        
                        if file_response.status_code == 200:
                            # Determine file extension based on content type
                            if 'application/vnd.ms-excel' in content_type:
                                extension = '.xls'
                            else:
                                extension = '.xlsx'
                            
                            # Update filename with correct extension
                            filename = f"migration_flows_{year}{extension}"
                            filepath = os.path.join(output_dir, filename)
                            
                            # Save the file
                            with open(filepath, 'wb') as f:
                                f.write(file_response.content)
                            
                            # Verify the file is a valid Excel file
                            try:
                                if extension == '.xls':
                                    df = pd.read_excel(filepath, engine='xlrd')
                                else:
                                    df = pd.read_excel(filepath, engine='openpyxl')
                                downloaded += 1
                                print(f"Successfully downloaded and verified {filename}")
                            except Exception as excel_error:
                                print(f"Downloaded file is not a valid Excel file: {str(excel_error)}")
                                # Remove invalid file
                                os.remove(filepath)
                        else:
                            print(f"Failed to download file for year {year}. Status code: {file_response.status_code}")
                    
                    except Exception as e:
                        print(f"Error downloading file for year {year}: {str(e)}")

    print(f"\nDownload complete. Successfully downloaded {downloaded} files, skipped {skipped} existing files.")

if __name__ == "__main__":
    download_migration_flows()

URL: https://www2.census.gov/programs-surveys/demo/tables/geographic-mobility/2023/state-to-state-migration/State_to_State_Migration_Table_2023_T13.xlsx
Content-Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
Successfully downloaded and verified migration_flows_2023.xlsx
URL: https://www2.census.gov/programs-surveys/demo/tables/geographic-mobility/2022/state-to-state-migration/State_to_State_Migration_Table_2022_T13_updated_2024_06_27.xlsx
Content-Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
Successfully downloaded and verified migration_flows_2022.xlsx
URL: https://www2.census.gov/programs-surveys/demo/tables/geographic-mobility/2021/state-to-state-migration/State_to_State_Migrations_Table_2021.xls
Content-Type: application/vnd.ms-excel
Successfully downloaded and verified migration_flows_2021.xls
URL: https://www2.census.gov/programs-surveys/demo/tables/geographic-mobility/2019/state-to-state-migration/State_to_State_Migrations_Table

In [9]:

def process_migration_file(file_path):
    try:
        if file_path.startswith('~$'):  # Skip temporary Excel files
            return None
            
        print(f"Processing file: {file_path}")
        # Read the Excel file
        df = pd.read_excel(file_path)
        
        # Get the year from the filename
        year = int(file_path.split('_')[-1].split('.')[0])
        print(f"Extracted year: {year}")
        
        # Drop the last rows (3 for pre-2010, 8 for 2010+)
        if year >= 2010:
            df = df.iloc[:-8].reset_index(drop=True)
        else:
            df = df.iloc[:-3].reset_index(drop=True)
        
        # Find columns that contain 'MOE' as a value and drop them
        moe_columns = []
        for col in df.columns:
            col_str = df[col].astype(str)
            if col_str.str.contains('MOE').any():
                moe_columns.append(col)
        df = df.drop(columns=moe_columns)
        
        # Drop the first 5 rows
        df = df.iloc[5:].reset_index(drop=True)
        
        # Drop rows where all values are missing
        df = df.dropna(how='all').reset_index(drop=True)
        
        # Drop specific rows
        try:
            if year >= 2010:
                df = df.drop(index=[1, 2, 30, 31, 32]).reset_index(drop=True)
            else:
                df = df.drop(index=[1, 32, 33, 34]).reset_index(drop=True)
        except KeyError:
            pass
        
        # Make the first row first column equal to 0
        df.iloc[0, 0] = 0
        
        # Set the first column as the index
        first_col = df.columns[0]
        df = df.set_index(first_col)
        
        # Drop columns where the first row has missing values
        df = df.loc[:, df.iloc[0].notna()]
        
        # Get states from the first row
        states = df.iloc[0].values
        
        # Create flows dataframe
        data = []
        for i in range(1, len(df)):
            for j in range(len(df.columns)):
                value = df.iloc[i, j]
                if pd.notna(value) and str(value).strip() != '':
                    try:
                        # Convert value to float, handling any commas
                        clean_value = str(value).replace(',', '')
                        float_value = float(clean_value)
                        data.append({
                            'From': states[j],
                            'To': df.index[i],
                            'Year': year,
                            'Population Change': float_value
                        })
                    except (ValueError, TypeError):
                        continue
        
        result_df = pd.DataFrame(data)
        print(f"Successfully processed {file_path}, created {len(result_df)} rows")
        return result_df
    
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

# Get all Excel files in the directory
excel_files = glob.glob('*.xls*')
print(f"Found {len(excel_files)} Excel files: {excel_files}")

# Process all files and store results in a list
all_flows = []
for file in excel_files:
    if not file.startswith('~$'):  # Skip temporary Excel files
        flows = process_migration_file(file)
        if flows is not None:
            all_flows.append(flows)

# Check if we have any processed data
if len(all_flows) > 0:
    # Combine all flows into one dataframe
    combined_flows = pd.concat(all_flows, ignore_index=True)
    print(f"Created combined dataset with {len(combined_flows)} rows")

    # Remove rows where From and To are the same
    combined_flows = combined_flows[combined_flows['From'] != combined_flows['To']]
    
    # Remove 'Total', 'U.S. Island Area', 'Foreign Country', 'Foreign Country4', 'U.S. Island Area3', and 'Puerto Rico'
    combined_flows = combined_flows[~combined_flows['From'].isin(['Total', 'U.S. Island Area', 'Foreign Country', 'Foreign Country4', 'U.S. Island Area3', 'Puerto Rico'])]
    # Remove 'Total', 'U.S. Island Area', 'Foreign Country', 'Foreign Country4', 'U.S. Island Area3'
    combined_flows = combined_flows[~combined_flows['To'].isin(['Total', 'U.S. Island Area', 'Foreign Country', 'Foreign Country4', 'U.S. Island Area3', 'Puerto Rico'])]


    # Save the combined results
    combined_flows.to_csv('migration_flows_combined.csv', index=False)
    print("Saved results to migration_flows_combined.csv")
else:
    print("No files were successfully processed!")

Found 18 Excel files: ['migration_flows_2005.xls', 'migration_flows_2006.xls', 'migration_flows_2007.xls', 'migration_flows_2008.xls', 'migration_flows_2009.xls', 'migration_flows_2010.xls', 'migration_flows_2011.xls', 'migration_flows_2012.xls', 'migration_flows_2013.xls', 'migration_flows_2014.xls', 'migration_flows_2015.xls', 'migration_flows_2016.xls', 'migration_flows_2017.xls', 'migration_flows_2018.xls', 'migration_flows_2019.xls', 'migration_flows_2021.xls', 'migration_flows_2022.xlsx', 'migration_flows_2023.xlsx']
Processing file: migration_flows_2005.xls
Extracted year: 2005
Successfully processed migration_flows_2005.xls, created 2704 rows
Processing file: migration_flows_2006.xls
Extracted year: 2006
Successfully processed migration_flows_2006.xls, created 2704 rows
Processing file: migration_flows_2007.xls
Extracted year: 2007
Successfully processed migration_flows_2007.xls, created 2704 rows
Processing file: migration_flows_2008.xls
Extracted year: 2008
Successfully proce